<a href="https://colab.research.google.com/github/sanchavali/Language-Detection/blob/main/Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### required libraries

In [ ]:
import pandas as pd
import numpy as np
import pycld2 as cld2
import pyodbc
import sys
import sqlalchemy
import urllib
from sqlalchemy import create_engine, event
import time

In [ ]:
cnxn=pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                          'Server=;'
                          'Database=RDR_SCRUB_DEV;'
                          'Trusted_Connection=yes;')

In [ ]:
cursor = cnxn.cursor()

In [ ]:
sqlquery = ('select FRST_ACTVY_ID, DPSCH_CMNTS from SC_RDR_RCA_FRST_DSPCH_CMNTS '
'where FRST_DSPCH_CMNTS_QLFY_FLG=1;')

In [ ]:
df=pd.read_sql_query(sqlquery,cnxn)

##### replacing Non-printable characters

In [ ]:
df["DPSCH_CMNTS1"] = df["DPSCH_CMNTS"].astype(str).apply(lambda x: ''.join([ch for ch in x if ch.isprintable()]))

##### Languge detection

In [ ]:
df['language']=df['DPSCH_CMNTS1'].apply(lambda x:[r[0] for r in cld2.detect(x)[2]])

In [ ]:
df['language']=df['language'].apply(tuple)

In [ ]:
column_sel = df[['FRST_ACTVY_ID','language']]
df_Ins=column_sel.copy()

In [ ]:
df_Ins['language']=df_Ins['language'].apply(tuple)

In [ ]:
df_Ins['language'] = df_Ins['language'].apply(lambda x: ', '.join(x) if isinstance(x,tuple) else x)

###### Insert data into sqlserver

In [ ]:
quoted = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=;DATABASE=RDR_SCRUB_DEV;Trusted_Connection=yes;")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

In [ ]:
#df_Ins.to_sql('SC_RDR_RCA_FRST_DPS_CMNTS_LANG', schema='dbo', con = engine, chunksize=200, method='multi', index=False, if_exists='replace')


@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True



df_Ins.to_sql("SC_RDR_RCA_FRST_DPS_CMNTS_LANG", con = engine, method='multi', if_exists='append', index=False)